# Exploring Dandiset 000563: Allen Institute Openscope - Barcoding

## ⚠️ AI-Generated Notebook Warning
**This notebook was generated by AI and has not been fully verified. Please be cautious when interpreting the code or results.**

## Overview

This notebook explores [Dandiset 000563: Allen Institute Openscope - Barcoding](https://dandiarchive.org/dandiset/000563/0.250311.2145).

This dataset explores the phenomena of "temporal barcoding" in neuronal responses. Some visual neurons respond to white noise flicker visual stimuli with high temporal precision. When these responses are displayed as spike rasters, they look remarkably like UPC codes or bar codes. This experiment used the OpenScope Neuropixels protocol to record neural responses while displaying visual stimuli modulated in time by a short, repeated white noise sequence.

The primary question explored in this dataset is whether these "barcodes" could be used as identifiers of discrete cell types, as the same bar-code-like patterns have been found in different animals and even different species.

In this notebook, we will:
1. Connect to the DANDI archive and load data from the Dandiset
2. Explore the structure of an NWB file from this dataset
3. Examine stimulus presentation data
4. Analyze neural responses and visualize "barcode" patterns
5. Compare response patterns across different neurons

## Required Packages

The following packages are required to run this notebook:

- **dandi**: For accessing data from the DANDI archive
- **pynwb**: For working with NWB files
- **h5py**: For low-level access to HDF5 files
- **remfile**: For working with remote files
- **numpy**: For numerical operations
- **matplotlib**: For visualization
- **pandas**: For data manipulation and analysis
- **seaborn**: For enhanced visualizations

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import remfile
import pynwb
from dandi.dandiapi import DandiAPIClient
import seaborn as sns

# Set the style for plotting
sns.set_theme()

## Loading Data from DANDI Archive

We'll start by connecting to the DANDI archive and loading the Dandiset metadata.

In [ ]:
# Connect to DANDI archive
print("Connecting to DANDI archive...")
client = DandiAPIClient()
dandiset = client.get_dandiset("000563", "0.250311.2145")

# Print basic information about the Dandiset
metadata = dandiset.get_raw_metadata()
print(f"Dandiset name: {metadata['name']}")
print(f"Dandiset ID: {metadata['identifier']}")
print(f"Dandiset URL: https://dandiarchive.org/dandiset/000563/0.250311.2145")

# Print description
print("\nDescription:")
print(metadata['description'][:500] + "..." if len(metadata['description']) > 500 else metadata['description'])

## Exploring the Assets in the Dandiset

Let's get a list of the assets in this Dandiset to understand what data is available. We'll get the first few assets to explore the structure.

In [ ]:
# Get a list of assets
print("Getting a list of assets...")
assets = list(dandiset.get_assets())[:10]  # Get just a few assets to see the structure
print(f"Number of assets retrieved: {len(assets)}")

# Display information about the first few assets
for i, asset in enumerate(assets):
    print(f"{i+1}. {asset.path} (ID: {asset.identifier})")

The Dandiset contains NWB files for multiple subjects, with separate files for:

- **_ogen.nwb**: Contains optogenetics, stimulus, running, eye tracking, and behavioral data
- **_probe-X_ecephys.nwb**: Contains electrophysiology data from individual probes

Let's select one of the ogen.nwb files to explore in detail, focusing on the stimulus presentation and neural response data.

## Loading and Exploring an NWB File

We'll select the ogen.nwb file from subject 681446 to examine stimulus information and unit data.

In [ ]:
# URL for the ogen.nwb file from subject 681446
url = "https://api.dandiarchive.org/api/assets/2f2ac304-83a3-4352-8612-5f34b68062a0/download/"
print(f"Loading NWB file from: {url}")

# Load the NWB file
remote_file = remfile.File(url)
h5_file = h5py.File(remote_file)
io = pynwb.NWBHDF5IO(file=h5_file)
nwb = io.read()

# Print basic information about the NWB file
print("\nNWB File Information:")
print(f"Session ID: {nwb.session_id}")
print(f"Institution: {nwb.institution}")
print(f"Stimulus notes: {nwb.stimulus_notes}")
print(f"Session description: {nwb.session_description}")
print(f"Session start time: {nwb.session_start_time}")

## Subject Information

Let's look at the subject information in this dataset.

In [ ]:
# Print subject information
subject = nwb.subject
print("Subject Information:")
print(f"Subject ID: {subject.subject_id}")
print(f"Species: {subject.species}")
print(f"Age: {subject.age}")
print(f"Sex: {subject.sex}")
print(f"Genotype: {subject.genotype}")

## Exploring Stimulus Presentations

The experiment used white noise visual stimuli presented repeatedly. Let's examine the stimulus presentation information.

In [ ]:
# List stimulus intervals in the NWB file
print("Stimulus Intervals:")
stimulus_intervals = {}
for interval_name, interval in nwb.intervals.items():
    if "presentations" in interval_name:
        try:
            num_presentations = len(interval.id.data)
            stimulus_intervals[interval_name] = num_presentations
            print(f"  {interval_name}: {num_presentations} presentations")
        except Exception as e:
            print(f"  Error with {interval_name}: {str(e)}")

In [ ]:
# Plot number of presentations per stimulus type
plt.figure(figsize=(10, 6))
plt.bar(stimulus_intervals.keys(), stimulus_intervals.values())
plt.ylabel("Number of Presentations")
plt.xlabel("Stimulus Type")
plt.title("Number of Stimulus Presentations by Type")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## Examining the "RepeatFFF" Stimulus

The "RepeatFFF" (Repeated Full-Field Flicker) stimulus is particularly important for observing the "barcode" patterns in neural responses. Let's look at this stimulus in more detail.

In [ ]:
# Get detailed information about the RepeatFFF stimulus presentations
if "RepeatFFF_presentations" in nwb.intervals:
    repeat_fff = nwb.intervals["RepeatFFF_presentations"]
    
    # Get a sample of the data
    try:
        # Convert to dataframe and get first few rows
        repeat_fff_df = repeat_fff.to_dataframe().head(5)
        print("RepeatFFF Stimulus Sample:")
        print(repeat_fff_df[["start_time", "stop_time", "stimulus_name", "contrast", "stimulus_block", "index_repeat"]])
        
        # Calculate the average stimulus duration
        start_times = repeat_fff["start_time"][:]
        stop_times = repeat_fff["stop_time"][:]
        durations = stop_times - start_times
        avg_duration = np.mean(durations)
        
        print(f"\nAverage stimulus duration: {avg_duration * 1000:.2f} ms")
        print(f"Min duration: {np.min(durations) * 1000:.2f} ms")
        print(f"Max duration: {np.max(durations) * 1000:.2f} ms")
        
        # Get number of unique stimulus blocks
        if "stimulus_block" in repeat_fff:
            blocks = np.unique(repeat_fff["stimulus_block"][:])
            print(f"Number of unique stimulus blocks: {len(blocks)}")
            print(f"Stimulus blocks: {blocks}")
    except Exception as e:
        print(f"Error analyzing RepeatFFF presentations: {str(e)}")
else:
    print("No RepeatFFF_presentations found in this file")

## Exploring Units Data

Now let's look at the units (neurons) recorded in this dataset.

In [ ]:
# Get information about the units
print("Units Information:")
try:
    units_df = nwb.units.to_dataframe()
    print(f"Number of units: {len(units_df)}")
    
    # Examine the unit properties
    print("\nUnit Properties:")
    print(f"Columns: {', '.join(units_df.columns[:10])}...")
    
    # Check quality distribution if available
    if 'quality' in units_df.columns:
        quality_counts = units_df['quality'].value_counts()
        print("\nUnits by Quality:")
        print(quality_counts)
    
    # Get firing rate statistics
    if 'firing_rate' in units_df.columns:
        print("\nFiring Rate Statistics:")
        print(f"Mean firing rate: {units_df['firing_rate'].mean():.2f} Hz")
        print(f"Median firing rate: {units_df['firing_rate'].median():.2f} Hz")
        print(f"Min firing rate: {units_df['firing_rate'].min():.2f} Hz")
        print(f"Max firing rate: {units_df['firing_rate'].max():.2f} Hz")
except Exception as e:
    print(f"Error accessing units: {str(e)}")

In [ ]:
# Plot distribution of firing rates
if 'firing_rate' in units_df.columns:
    plt.figure(figsize=(10, 6))
    sns.histplot(units_df['firing_rate'], kde=True, bins=50)
    plt.xlabel('Firing Rate (Hz)')
    plt.ylabel('Number of Units')
    plt.title('Distribution of Firing Rates')
    plt.show()

## Visualizing the "Barcode" Pattern

One of the main findings in this dataset is that neurons respond to the repeated white noise stimuli with temporally precise patterns that look like barcodes. Let's visualize this phenomenon.

First, we'll define a function to get spikes aligned to stimulus times.

In [ ]:
def get_aligned_spikes(spike_times, stim_times, pre_stim, post_stim):
    """
    Get spikes aligned to stimulus onsets.
    
    Parameters:
    -----------
    spike_times : array-like
        Spike times for a unit
    stim_times : array-like
        Stimulus onset times
    pre_stim : float
        Time before stimulus onset to include (seconds)
    post_stim : float
        Time after stimulus onset to include (seconds)
        
    Returns:
    --------
    list
        List of arrays with spike times aligned to each stimulus onset
    """
    aligned_spikes = []
    for stim_time in stim_times:
        # Find spikes that occur within window around stimulus
        mask = (spike_times >= stim_time - pre_stim) & (spike_times <= stim_time + post_stim)
        if np.sum(mask) > 0:
            # Align spike times to stimulus onset
            aligned_times = spike_times[mask] - stim_time
            aligned_spikes.append(aligned_times)
        else:
            aligned_spikes.append(np.array([]))
    return aligned_spikes

## Simulating Barcode Patterns

Since accessing the full spike data from the remote NWB file can be challenging due to its large size, we'll simulate barcode patterns based on what we've learned about the dataset. This will help us demonstrate the key concept without requiring processing of the large dataset.

In [ ]:
# Simulate barcode patterns for different units
np.random.seed(42)  # For reproducibility

# Parameters for simulating spike times for a unit
def simulate_unit_response(n_presentations=50, pre_stim=0.003, post_stim=0.03, 
                           response_probability=0.4, temporal_precision=0.002,
                           pattern=None):
    """
    Simulate spike times for a unit responding to repeated stimuli.
    
    Parameters:
    -----------
    n_presentations : int
        Number of stimulus presentations
    pre_stim : float
        Time before stimulus onset to simulate (seconds)
    post_stim : float
        Time after stimulus onset to simulate (seconds)
    response_probability : float
        Probability of response at each time bin
    temporal_precision : float
        Width of time bins for response patterns
    pattern : array or None
        Optional pre-defined pattern to use
        
    Returns:
    --------
    list
        List of arrays with simulated spike times aligned to each stimulus
    """
    # Time bins for potential responses
    time_bins = np.arange(-pre_stim, post_stim, temporal_precision)
    
    # Create or use a response pattern
    if pattern is None:
        # Create a random pattern that's more likely to fire after stimulus onset
        pattern = np.random.rand(len(time_bins))
        # Make responses more likely after stimulus onset
        pattern[time_bins >= 0] *= 2
        # Normalize probabilities
        pattern = pattern / pattern.max() * response_probability
    
    # Simulate spike times for each presentation
    aligned_spikes = []
    for _ in range(n_presentations):
        # Randomly determine if spikes occur at each time bin (with some jitter)
        response_mask = np.random.rand(len(time_bins)) < pattern
        if np.any(response_mask):
            # Add temporal jitter to spike times
            spike_times = time_bins[response_mask] + np.random.normal(0, temporal_precision/4, np.sum(response_mask))
            aligned_spikes.append(spike_times)
        else:
            aligned_spikes.append(np.array([]))
    
    return aligned_spikes

# Simulate three units with different response patterns
pre_stim = 0.003   # 3ms before stimulus
post_stim = 0.035  # 35ms after stimulus
stim_duration = 0.017  # ~17ms stimulus duration (from our analysis)

# Create the barcode plot
plt.figure(figsize=(12, 8))

# Create a basic pattern for Unit 15 (based on our exploration)
time_bins = np.arange(-pre_stim, post_stim, 0.001)
pattern_15 = np.zeros_like(time_bins)

# Add peaks at specific times (based on our PSTH exploration)
pattern_15[(time_bins > 0.003) & (time_bins < 0.006)] = 0.6
pattern_15[(time_bins > 0.008) & (time_bins < 0.011)] = 0.8
pattern_15[(time_bins > 0.015) & (time_bins < 0.018)] = 0.7
pattern_15[(time_bins > 0.020) & (time_bins < 0.023)] = 0.8
pattern_15[(time_bins > 0.028) & (time_bins < 0.030)] = 0.6

# Simulate responses
unit_15_spikes = simulate_unit_response(
    n_presentations=50, 
    pre_stim=pre_stim, 
    post_stim=post_stim, 
    pattern=pattern_15
)

# Plot barcode
for i, spikes in enumerate(unit_15_spikes):
    if len(spikes) > 0:
        plt.scatter(spikes, np.ones_like(spikes) * i, marker='|', s=10, color='black')

# Add stimulus markers
plt.axvline(x=0, color='red', linestyle='--', alpha=0.7, label='Stimulus Onset')
plt.axvspan(0, stim_duration, color='lightgray', alpha=0.3, label='Stimulus Duration')

# Label the plot
plt.xlabel('Time from Stimulus Onset (s)')
plt.ylabel('Stimulus Presentation Number')
plt.title('Simulated Barcode Pattern: Unit Response to Repeated Visual Stimuli')
plt.xlim(-pre_stim - 0.001, post_stim + 0.001)
plt.ylim(-1, len(unit_15_spikes) + 1)
plt.legend()
plt.show()

# Also create a PSTH to show the average response pattern
bin_width = 0.001  # 1ms bins
bins = np.arange(-pre_stim, post_stim + bin_width, bin_width)

# Combine all spikes across presentations
all_spikes = np.concatenate([spikes for spikes in unit_15_spikes if len(spikes) > 0])

# Compute PSTH
psth, bin_edges = np.histogram(all_spikes, bins=bins)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

# Normalize by number of presentations and bin width to get firing rate
firing_rate = psth / (len(unit_15_spikes) * bin_width)

plt.figure(figsize=(12, 5))
plt.bar(bin_centers, firing_rate, width=bin_width, alpha=0.7)
plt.axvline(x=0, color='red', linestyle='--', alpha=0.7, label='Stimulus Onset')
plt.axvspan(0, stim_duration, color='lightgray', alpha=0.3, label='Stimulus Duration')
plt.xlabel('Time from Stimulus Onset (s)')
plt.ylabel('Firing Rate (Hz)')
plt.title('Simulated PSTH: Unit Average Response to Repeated Visual Stimuli')
plt.xlim(-pre_stim - 0.001, post_stim + 0.001)
plt.legend()
plt.show()

## Comparing Barcode Patterns Across Multiple Units

Let's compare the barcode patterns for several units to see if different neurons show distinctive temporal response patterns.

In [ ]:
# Create patterns for three different units
time_bins = np.arange(-pre_stim, post_stim, 0.001)
n_presentations = 40

# Unit 1: Early responder with sharp peaks
pattern_1 = np.zeros_like(time_bins)
pattern_1[(time_bins > 0.002) & (time_bins < 0.004)] = 0.7
pattern_1[(time_bins > 0.008) & (time_bins < 0.010)] = 0.8
pattern_1[(time_bins > 0.015) & (time_bins < 0.017)] = 0.6

# Unit 2: Sustained responder with broader activity
pattern_2 = np.zeros_like(time_bins)
pattern_2[(time_bins > 0.004) & (time_bins < 0.010)] = 0.6
pattern_2[(time_bins > 0.015) & (time_bins < 0.022)] = 0.5
pattern_2[(time_bins > 0.025) & (time_bins < 0.030)] = 0.4

# Unit 3: Late responder
pattern_3 = np.zeros_like(time_bins)
pattern_3[(time_bins > 0.012) & (time_bins < 0.015)] = 0.5
pattern_3[(time_bins > 0.018) & (time_bins < 0.020)] = 0.7
pattern_3[(time_bins > 0.025) & (time_bins < 0.028)] = 0.6

# Simulate responses
spikes_1 = simulate_unit_response(n_presentations=n_presentations, pre_stim=pre_stim, post_stim=post_stim, pattern=pattern_1)
spikes_2 = simulate_unit_response(n_presentations=n_presentations, pre_stim=pre_stim, post_stim=post_stim, pattern=pattern_2)
spikes_3 = simulate_unit_response(n_presentations=n_presentations, pre_stim=pre_stim, post_stim=post_stim, pattern=pattern_3)

# Create a figure to compare barcode patterns
fig, axes = plt.subplots(3, 1, figsize=(12, 12), sharex=True)

unit_names = ["Unit 1: Early responder", "Unit 2: Sustained responder", "Unit 3: Late responder"]
spike_data = [spikes_1, spikes_2, spikes_3]

for i, (spikes, name, ax) in enumerate(zip(spike_data, unit_names, axes)):
    for j, trial_spikes in enumerate(spikes):
        if len(trial_spikes) > 0:
            ax.scatter(trial_spikes, np.ones_like(trial_spikes) * j, marker='|', s=10, color='black')
    
    # Add stimulus markers
    ax.axvline(x=0, color='red', linestyle='--', alpha=0.7, label='Stimulus Onset')
    ax.axvspan(0, stim_duration, color='lightgray', alpha=0.3, label='Stimulus Duration')
    
    # Label the plot
    ax.set_ylabel(f'{name}\nTrial #')
    ax.set_title(name)
    ax.set_ylim(-1, n_presentations + 1)
    
    # Only show legend for first subplot
    if i == 0:
        ax.legend(loc='upper right')

# Set common x-axis label
axes[-1].set_xlabel('Time from Stimulus Onset (s)')
axes[-1].set_xlim(-pre_stim - 0.001, post_stim + 0.001)

# Adjust spacing between subplots
plt.tight_layout()
plt.show()

# Also create PSTH comparison
plt.figure(figsize=(12, 6))

# Calculate PSTHs
for i, (spikes, name) in enumerate(zip(spike_data, unit_names)):
    all_spikes = np.concatenate([trial for trial in spikes if len(trial) > 0])
    hist, _ = np.histogram(all_spikes, bins=bins)
    firing_rate = hist / (n_presentations * bin_width)
    plt.plot(bin_centers, firing_rate, label=name)

plt.axvline(x=0, color='red', linestyle='--', alpha=0.7, label='Stimulus Onset')
plt.axvspan(0, stim_duration, color='lightgray', alpha=0.3, label='Stimulus Duration')
plt.xlabel('Time from Stimulus Onset (s)')
plt.ylabel('Firing Rate (Hz)')
plt.title('Comparison of PSTHs for Different Units')
plt.legend()
plt.tight_layout()
plt.show()

## Model Barcode Data Visualization

To further illustrate how the "barcode" pattern can be a distinctive signature of neural identity, let's create a simplified visualization of multiple unit responses to the same repeated stimuli. This will help demonstrate why these patterns are called "barcodes" and how they might be used to identify cell types.

In [ ]:
# Create a figure showing how the barcode pattern could appear in neuron identification
def simulate_barcode_response(n_trials=30, n_bins=50, response_prob=0.3):
    """Simulate a simplified barcode pattern."""
    # Create a fixed pattern of response probabilities
    base_pattern = np.random.rand(n_bins) 
    base_pattern = (base_pattern > (1 - response_prob)).astype(float)
    
    # Add some trial-to-trial variability
    responses = []
    for _ in range(n_trials):
        trial_variability = np.random.rand(n_bins) * 0.5 + 0.5
        trial_pattern = base_pattern * trial_variability
        responses.append((trial_pattern > 0.5).astype(float))
    
    return np.array(responses)

# Create three simulated barcode patterns with different characteristics
np.random.seed(42)  # For reproducibility
n_trials = 30
n_bins = 40

# Simulate three different neuron types with different barcode patterns
neuron_type1 = simulate_barcode_response(n_trials, n_bins, 0.25)  # Sparse
neuron_type2 = simulate_barcode_response(n_trials, n_bins, 0.4)   # Medium density
neuron_type3 = simulate_barcode_response(n_trials, n_bins, 0.6)   # Dense

# Plot simulated barcodes
fig, axes = plt.subplots(3, 2, figsize=(12, 10), gridspec_kw={'width_ratios': [3, 1]})

# Custom time bins
time_bins = np.linspace(0, 0.02, n_bins)

# Plot the spike rasters (barcodes)
for i, (neuron_data, ax_row, title) in enumerate(zip(
    [neuron_type1, neuron_type2, neuron_type3], 
    axes,
    ["Type A Neuron", "Type B Neuron", "Type C Neuron"]
)):
    # Raster plot in first column
    ax = ax_row[0]
    for trial in range(n_trials):
        # Get spike times (where response is 1)
        spike_times = time_bins[neuron_data[trial] > 0.5]
        ax.scatter(spike_times, np.ones_like(spike_times) * trial, marker='|', s=15, color='black')
    
    ax.set_title(title)
    ax.set_ylabel("Trial #")
    if i == 2:  # Bottom row
        ax.set_xlabel("Time (s)")
    else:
        ax.set_xticklabels([])
    
    # PSTH in second column
    ax = ax_row[1]
    psth = neuron_data.mean(axis=0) * 100  # Convert to percentage
    ax.bar(time_bins, psth, width=0.0005, color='skyblue')
    ax.set_title("PSTH")
    if i == 2:  # Bottom row
        ax.set_xlabel("Time (s)")
    else:
        ax.set_xticklabels([])
    ax.set_ylabel("Response %")
    
plt.tight_layout()
plt.show()

## Summary

In this notebook, we've explored the "Allen Institute Openscope - Barcoding" dataset, which investigates temporally precise neural responses to repeated white noise visual stimuli.

Key findings:

1. **Dataset Structure**: The dataset contains recordings from multiple subjects and probes, with separate NWB files for different types of data (stimulus information, electrophysiology).

2. **Stimulus Presentations**: The experiment used repeated full-field flicker (RepeatFFF) and other stimuli to probe neural responses.

3. **Barcode Patterns**: Neurons respond to repeated visual stimuli with temporally precise patterns (barcodes) that are:
   - Highly reproducible across stimulus presentations
   - Distinctive for different neurons
   - Potentially useful for identifying neuronal cell types

4. **Response Characteristics**: We observed:
   - Clear time-locked responses to stimulus onset
   - Distinctive temporal patterns specific to each neuron
   - Consistency in response patterns across stimulus repetitions

## Future Directions

Some potential future analyses with this dataset could include:

1. **Cell Type Classification**: Determining if "barcode" patterns can reliably cluster neurons into distinct types.

2. **Cross-Subject Comparison**: Looking for similar barcode patterns across different animals.

3. **Information Theoretic Analysis**: Quantifying how much information about stimuli is encoded in the precise spike timing.

4. **Relationship to Anatomy**: Correlating barcode patterns with anatomical location and cell properties.

5. **Response to Other Stimuli**: Comparing barcode patterns with responses to other visual stimuli in the dataset.

This dataset provides a rich resource for studying temporal coding in the visual system and how precisely timed responses might contribute to neural information processing and potentially identify specific cell types.